In [1]:
import os
import pretty_midi
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
from hvo_sequence import HVO_Sequence
from hvo_sequence import ROLAND_REDUCED_MAPPING
from hvo_sequence import io_helpers
import midi_helper
import groove_midi as groove_midi

/opt/anaconda3/envs/TripleStreams/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
def combine_four_hvo_to_single_hvo(mapping, hs_individual_streams_list):

    #We set each voice to a Midi Mapping value for Drums
    #Four_Voices = {
    #    "voice_1": [36],
    #    "voice_2": [37],
    #    "voice_3": [38],
    #    "voice_4": [39]
    #}

    #Create new HVO Sequence with beat division of 4 and set Drum Mapping to Four_Voices Mapping
    hvo_seq_all_voices = HVO_Sequence(
        beat_division_factors=[4],
        drum_mapping=mapping)

    #Add tempo of 120 BPM to the HVO sequence at time 0
    hvo_seq_all_voices.add_tempo(
        time_step=0,
        qpm=120
    )

    #Add a time signature of 4/4 for the HVO Sequence
    hvo_seq_all_voices.add_time_signature(
        time_step=0,
        numerator=4,
        denominator=4
    )

    #Adjust the length of the HVO Sequence to 
    hvo_seq_all_voices.adjust_length(hs_individual_streams_list[0].hvo.shape[0])

    #print (hvo_seq_all_voices.hvo.shape)

    for i in range(4):
        hvo_seq_all_voices.hvo[:, i] = hs_individual_streams_list[i].hvo[:, 0]
        hvo_seq_all_voices.hvo[:, i+4] = hs_individual_streams_list[i].hvo[:, 1]
        hvo_seq_all_voices.hvo[:, i+8] = hs_individual_streams_list[i].hvo[:, 2]

    return hvo_seq_all_voices

In [14]:
error_count = 0
for root, _, files in os.walk('groove_midi/groove_merged'):
    for file in files:
        if file.endswith('.mid'):
            print(file)
            pm_testing_merge = pretty_midi.PrettyMIDI(os.path.join(root, file))
            if 'functional_hats_crash' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [49, 52, 55, 57]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_hats_ride' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [51, 53, 59]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_hats_hh_closed' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [42, 22, 44]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_hats_hh_open' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [46, 26]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1

            elif 'functional_kick' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [36, 35]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_toms' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [43, 58, 41, 47, 45, 50, 48]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_snare' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [38, 37, 40, 39]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'functional_hh.mid' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [42, 22, 44, 46, 26]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1

            elif 'pitch_low' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [35, 36, 43, 58, 41]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'pitch_mid' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [38, 37, 40, 39, 47, 45, 50, 48]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'pitch_high' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [42, 22, 44, 46, 26, 49, 52, 55, 57, 51, 53, 59]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
           
            elif 'velocity_low' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.velocity >= 43:
                        print(f"Unexpected velocity {notes.velocity} in {file}")
                        error_count += 1
            elif 'velocity_mid' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.velocity < 43 or notes.velocity > 70:
                        print(f"Unexpected velocity {notes.velocity} in {file}")
                        error_count += 1
            elif 'velocity_high' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.velocity <= 70:
                        print(f"Unexpected velocity {notes.velocity} in {file}")
                        error_count += 1
            
            elif 'toms_lo' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [43, 58, 41]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'toms_mid' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [47, 45]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'toms_hi' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [50, 48]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
            elif 'toms_ride' in file:
                for notes in pm_testing_merge.instruments[0].notes:
                    if notes.pitch not in [51, 53, 59]:
                        print(f"Unexpected pitch {notes.pitch} in {file}")
                        error_count += 1
print(error_count)

113_afrocuban-bembe_122_fill_4-4_velocity_mid.mid
113_afrocuban-bembe_122_fill_4-4_velocity_low.mid
113_afrocuban-bembe_122_fill_4-4_velocity_groove.mid
113_afrocuban-bembe_122_fill_4-4_velocity_high.mid
20_latin-brazilian-baiao_110_beat_4-4_functionalhats_hhopen.mid
20_latin-brazilian-baiao_110_beat_4-4_pitch_high.mid
20_latin-brazilian-baiao_110_beat_4-4_functional_snare.mid
20_latin-brazilian-baiao_110_beat_4-4_velocity_low.mid
20_latin-brazilian-baiao_110_beat_4-4_velocity_mid.mid
20_latin-brazilian-baiao_110_beat_4-4_pitch_groove.mid
20_latin-brazilian-baiao_110_beat_4-4_toms_lo.mid
20_latin-brazilian-baiao_110_beat_4-4_toms_ride.mid
20_latin-brazilian-baiao_110_beat_4-4_velocity_high.mid
20_latin-brazilian-baiao_110_beat_4-4_functional_kick.mid
20_latin-brazilian-baiao_110_beat_4-4_functionalhats_crash.mid
20_latin-brazilian-baiao_110_beat_4-4_functionalhats_ride.mid
20_latin-brazilian-baiao_110_beat_4-4_functional_toms.mid
20_latin-brazilian-baiao_110_beat_4-4_pitch_mid.mid
20_l

In [72]:
from pickle import load
hvo_seq_loaded_1 = load(open("groove_midi/groove_hvo/1_latin-chacarera_157_beat_3-4_velocity/1_latin-chacarera_157_beat_3-4_velocity_groove.hvo", "rb"))
hvo_seq_loaded_2 = load(open("groove_midi/groove_hvo/1_latin-chacarera_157_beat_3-4_velocity/1_latin-chacarera_157_beat_3-4_velocity_high.hvo", "rb"))
hvo_seq_loaded_3 = load(open("groove_midi/groove_hvo/1_latin-chacarera_157_beat_3-4_velocity/1_latin-chacarera_157_beat_3-4_velocity_low.hvo", "rb"))
hvo_seq_loaded_4 = load(open("groove_midi/groove_hvo/1_latin-chacarera_157_beat_3-4_velocity/1_latin-chacarera_157_beat_3-4_velocity_mid.hvo", "rb"))

In [73]:
from bokeh.io import output_notebook, show
output_notebook()
hvo_seq_loaded_1.to_html_plot(
    filename="hvo_1_lakh.html",
    save_figure=False,
    show_figure=True)

Loading BokehJS ...

figure(id='p7572', ...)

In [74]:
from bokeh.io import output_notebook, show
output_notebook()
hvo_seq_loaded_2.to_html_plot(
    filename="hvo_1_lakh.html",
    save_figure=False,
    show_figure=True)

Loading BokehJS ...

figure(id='p9316', ...)

In [75]:
from bokeh.io import output_notebook, show
output_notebook()
hvo_seq_loaded_3.to_html_plot(
    filename="hvo_1_lakh.html",
    save_figure=False,
    show_figure=True)

Loading BokehJS ...

figure(id='p11059', ...)

In [77]:
from bokeh.io import output_notebook, show
output_notebook()
hvo_seq_loaded_4.to_html_plot(
    filename="hvo_4_lakh.html",
    save_figure=False,
    show_figure=True)

Loading BokehJS ...

figure(id='p14533', ...)

In [60]:
pm_test_bpm = pretty_midi.PrettyMIDI("groove_midi/groove_merged/188_reggae_78_fill_4-4/188_reggae_78_fill_4-4_velocity_groove.mid")
pm_test_bpm.get_tempo_changes()

(array([0.]), array([78.0001794]))